In [29]:
from ExploreData import ExploreData
import numpy as np
import pandas as pd
import os

In [30]:
print(os.getcwd())
data_path = '../data/'
ml_path = 'C:/Users/BrandonBrown/data/basketball/BB/ml/'

df_start = pd.read_csv(data_path + 'data_timeseries.csv')

display(df_start.sample(5).sort_index())
print('shape:', df_start.shape)

c:\Users\BrandonBrown\repos\basketball\scripts


,GAME_DATE_EST,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,season,days_since_last_game,season_short,percentage_regular_season_games,next_0_HOME,next_0_days_until_next_game,next_0_PTS_ALLOWED,next_0_FG_PCT_ALLOWED,next_0_FG3_PCT_ALLOWED,next_0_AST_ALLOWED,next_0_REB_ALLOWED,next_1_HOME,next_1_days_until_next_game,next_1_PTS_ALLOWED,next_1_FG_PCT_ALLOWED,next_1_FG3_PCT_ALLOWED,next_1_AST_ALLOWED,next_1_REB_ALLOWED,next_2_HOME,next_2_days_until_next_game,next_2_PTS_ALLOWED,next_2_FG_PCT_ALLOWED,next_2_FG3_PCT_ALLOWED,next_2_AST_ALLOWED,next_2_REB_ALLOWED,next_3_HOME,next_3_days_until_next_game,next_3_PTS_ALLOWED,next_3_FG_PCT_ALLOWED,next_3_FG3_PCT_ALLOWED,next_3_AST_ALLOWED,next_3_REB_ALLOWED,next_4_HOME,next_4_days_until_next_game,next_4_PTS_ALLOWED,next_4_FG_PCT_ALLOWED,next_4_FG3_PCT_ALLOWED,next_4_AST_ALLOWED,next_4_REB_ALLOWED,lag_1_START_POSITION,lag_1_comment_code,lag_1_comment_reason,lag_1_MIN,lag_1_FGM_pm,lag_1_FGA_pm,lag_1_FG3M_pm,lag_1_FG3A_pm,lag_1_FTM_pm,lag_1_FTA_pm,lag_1_REB_pm,lag_1_AST_pm,lag_1_STL_pm,lag_1_BLK_pm,lag_1_TO_pm,lag_1_PTS_pm,lag_1_PLUS_MINUS_pm,lag_1_PF_pm,lag_1_game_played,next_0_MIN,next_1_MIN,next_2_MIN,next_3_MIN,next_4_MIN,next_0_FGM,next_1_FGM,next_2_FGM,next_3_FGM,next_4_FGM,next_0_FGA,next_1_FGA,next_2_FGA,next_3_FGA,next_4_FGA,next_0_FG3M,next_1_FG3M,next_2_FG3M,next_3_FG3M,next_4_FG3M,next_0_FG3A,next_1_FG3A,next_2_FG3A,next_3_FG3A,next_4_FG3A,next_0_FTM,next_1_FTM,next_2_FTM,next_3_FTM,next_4_FTM,next_0_FTA,next_1_FTA,next_2_FTA,next_3_FTA,next_4_FTA,next_0_REB,next_1_REB,next_2_REB,next_3_REB,next_4_REB,next_0_AST,next_1_AST,next_2_AST,next_3_AST,next_4_AST,next_0_STL,next_1_STL,next_2_STL,next_3_STL,next_4_STL,next_0_BLK,next_1_BLK,next_2_BLK,next_3_BLK,next_4_BLK,next_0_TO,next_1_TO,next_2_TO,next_3_TO,next_4_TO,next_0_PTS,next_1_PTS,next_2_PTS,next_3_PTS,next_4_PTS,next_0_PLUS_MINUS,next_1_PLUS_MINUS,next_2_PLUS_MINUS,next_3_PLUS_MINUS,next_4_PLUS_MINUS,next_0_PF,next_1_PF,next_2_PF,next_3_PF,next_4_PF,avg_2_3_MIN,avg_4_9_MIN,avg_10_19_MIN,avg_20_35_MIN,avg_36_85_MIN,avg_2_3_FGM_pm,avg_4_9_FGM_pm,avg_10_19_FGM_pm,avg_20_35_FGM_pm,avg_36_85_FGM_pm,avg_2_3_FGA_pm,avg_4_9_FGA_pm,avg_10_19_FGA_pm,avg_20_35_FGA_pm,avg_36_85_FGA_pm,avg_2_3_FG3M_pm,avg_4_9_FG3M_pm,avg_10_19_FG3M_pm,avg_20_35_FG3M_pm,avg_36_85_FG3M_pm,avg_2_3_FG3A_pm,avg_4_9_FG3A_pm,avg_10_19_FG3A_pm,avg_20_35_FG3A_pm,avg_36_85_FG3A_pm,avg_2_3_FTM_pm,avg_4_9_FTM_pm,avg_10_19_FTM_pm,avg_20_35_FTM_pm,avg_36_85_FTM_pm,avg_2_3_FTA_pm,avg_4_9_FTA_pm,avg_10_19_FTA_pm,avg_20_35_FTA_pm,avg_36_85_FTA_pm,avg_2_3_REB_pm,avg_4_9_REB_pm,avg_10_19_REB_pm,avg_20_35_REB_pm,avg_36_85_REB_pm,avg_2_3_AST_pm,avg_4_9_AST_pm,avg_10_19_AST_pm,avg_20_35_AST_pm,avg_36_85_AST_pm,avg_2_3_STL_pm,avg_4_9_STL_pm,avg_10_19_STL_pm,avg_20_35_STL_pm,avg_36_85_STL_pm,avg_2_3_BLK_pm,avg_4_9_BLK_pm,avg_10_19_BLK_pm,avg_20_35_BLK_pm,avg_36_85_BLK_pm,avg_2_3_TO_pm,avg_4_9_TO_pm,avg_10_19_TO_pm,avg_20_35_TO_pm,avg_36_85_TO_pm,avg_2_3_PTS_pm,avg_4_9_PTS_pm,avg_10_19_PTS_pm,avg_20_35_PTS_pm,avg_36_85_PTS_pm,avg_2_3_PLUS_MINUS_pm,avg_4_9_PLUS_MINUS_pm,avg_10_19_PLUS_MINUS_pm,avg_20_35_PLUS_MINUS_pm,avg_36_85_PLUS_MINUS_pm,avg_2_3_PF_pm,avg_4_9_PF_pm,avg_10_19_PF_pm,avg_20_35_PF_pm,avg_36_85_PF_pm,avg_2_3_game_played,avg_4_9_game_played,avg_10_19_game_played,avg_20_35_game_played,avg_36_85_game_played,avg_1_20_played_in_first_game_of_back_to_back,avg_21_90_played_in_first_game_of_back_to_back,avg_1_20_played_in_second_game_of_back_to_back,avg_21_90_played_in_second_game_of_back_to_back,lag_1_MIN_season,lag_2_MIN_season,lag_1_game_played_season,lag_2_game_played_season,lag_1_FGM_pm_season,lag_2_FGM_pm_season,lag_1_FGA_pm_season,lag_2_FGA_pm_season,lag_1_FG3M_pm_season,lag_2_FG3M_pm_season,lag_1_FG3A_pm_season,lag_2_FG3A_pm_season,lag_1_FTM_pm_season,lag_2_FTM_pm_season,lag_1_FTA_pm_season,lag_2_FTA_pm_season,lag_1_REB_pm_season,lag_2_REB_pm_season,lag_1_AST_pm_season,lag_2_AST_pm_season,lag_1_STL_pm_season,lag_2_STL_pm_season,lag_1_BLK_pm_season,lag_2_BLK_pm_season,lag_1_TO_

shape: (579662, 276)


In [31]:
## UNCOMMENT TO FIND A SPECIFIC PLAYER IF STARTED IN LEAGUE AFTER 2019
# ## mask - filter for 2021 season and detroit pistons and did not play in 2021
# mask = (df_start['season_short'] == 2022) & (df_start['TEAM_ID'] == 1610612765) & (df_start['lag_1_MIN_season'].isna())
# df = df_start[mask].copy()
# df = df.sort_values(by=['GAME_DATE_EST'])

# ## display 300 columns and 82 rows
# pd.set_option('display.max_columns', 300)
# pd.set_option('display.max_rows', 82)

# # df where player id not in [1630587, 1630568, 1630296, 1630587, 1630691]
# # df[~df['PLAYER_ID'].isin([1630587, 1630568, 1630296, 1630587, 1630691])]

# ## player ids 1630587, 1630568, 1630296, 1630587, 1630691
# columns_with_next_0 = [x for x in df.columns if 'next_0' in x and 'ALLOWED' not in x]
# columns_with_next_0 = ['next_0_MIN', 'next_0_PTS', 'next_0_REB', 'next_0_AST']
# df.loc[df['PLAYER_ID'] == 1630595, ['GAME_DATE_EST', 'PLAYER_ID'] + columns_with_next_0]

In [32]:
## Impute missing values in labels with 0
index_cols = ['GAME_DATE_EST', 'GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'PLAYER_NAME', 'season']
all_labels = [
    'next_0_MIN', 'next_1_MIN', 'next_2_MIN', 'next_3_MIN', 'next_4_MIN',
    'next_0_PTS', 'next_1_PTS', 'next_2_PTS', 'next_3_PTS', 'next_4_PTS',
    'next_0_FGM', 'next_1_FGM', 'next_2_FGM', 'next_3_FGM', 'next_4_FGM',
    'next_0_FGA', 'next_1_FGA', 'next_2_FGA', 'next_3_FGA', 'next_4_FGA',
    'next_0_FTM', 'next_1_FTM', 'next_2_FTM', 'next_3_FTM', 'next_4_FTM',
    'next_0_FTA', 'next_1_FTA', 'next_2_FTA', 'next_3_FTA', 'next_4_FTA',
    'next_0_REB', 'next_1_REB', 'next_2_REB', 'next_3_REB', 'next_4_REB',
    'next_0_AST', 'next_1_AST', 'next_2_AST', 'next_3_AST', 'next_4_AST',
    'next_0_STL', 'next_1_STL', 'next_2_STL', 'next_3_STL', 'next_4_STL',
    'next_0_BLK', 'next_1_BLK', 'next_2_BLK', 'next_3_BLK', 'next_4_BLK',
    'next_0_TO', 'next_1_TO', 'next_2_TO', 'next_3_TO', 'next_4_TO',
    ]
do_not_include = [
    'next_0_FG3M', 'next_1_FG3M', 'next_2_FG3M', 'next_3_FG3M', 'next_4_FG3M',
    'next_0_FG3A', 'next_1_FG3A', 'next_2_FG3A', 'next_3_FG3A', 'next_4_FG3A',
    'next_0_PLUS_MINUS', 'next_1_PLUS_MINUS', 'next_2_PLUS_MINUS', 'next_3_PLUS_MINUS', 'next_4_PLUS_MINUS',
    'next_0_PF', 'next_1_PF', 'next_2_PF', 'next_3_PF', 'next_4_PF'
]
created_game_lag_cols = ['lag_1_START_POSITION', 'lag_1_comment_code', 'lag_1_comment_reason']
created_window_lag_cols = [
    'lag_1_MIN', 'lag_1_FGM_pm', 'lag_1_FGA_pm', 'lag_1_FG3M_pm', 'lag_1_FG3A_pm', 'lag_1_FTM_pm',
    'lag_1_FTA_pm', 'lag_1_REB_pm', 'lag_1_AST_pm', 'lag_1_STL_pm', 'lag_1_BLK_pm', 'lag_1_TO_pm',
    'lag_1_PTS_pm', 'lag_1_PLUS_MINUS_pm', 'lag_1_PF_pm', 'lag_1_game_played']
created_window_avg_cols = [
    'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN',
    'avg_2_3_FGM_pm', 'avg_4_9_FGM_pm', 'avg_10_19_FGM_pm', 'avg_20_35_FGM_pm', 'avg_36_85_FGM_pm',
    'avg_2_3_FGA_pm', 'avg_4_9_FGA_pm', 'avg_10_19_FGA_pm', 'avg_20_35_FGA_pm', 'avg_36_85_FGA_pm',
    'avg_2_3_FG3M_pm', 'avg_4_9_FG3M_pm', 'avg_10_19_FG3M_pm', 'avg_20_35_FG3M_pm', 'avg_36_85_FG3M_pm',
    'avg_2_3_FG3A_pm', 'avg_4_9_FG3A_pm', 'avg_10_19_FG3A_pm', 'avg_20_35_FG3A_pm', 'avg_36_85_FG3A_pm',
    'avg_2_3_FTM_pm', 'avg_4_9_FTM_pm', 'avg_10_19_FTM_pm', 'avg_20_35_FTM_pm', 'avg_36_85_FTM_pm',
    'avg_2_3_FTA_pm', 'avg_4_9_FTA_pm', 'avg_10_19_FTA_pm', 'avg_20_35_FTA_pm', 'avg_36_85_FTA_pm',
    'avg_2_3_REB_pm', 'avg_4_9_REB_pm', 'avg_10_19_REB_pm', 'avg_20_35_REB_pm', 'avg_36_85_REB_pm',
    'avg_2_3_AST_pm', 'avg_4_9_AST_pm', 'avg_10_19_AST_pm', 'avg_20_35_AST_pm', 'avg_36_85_AST_pm',
    'avg_2_3_STL_pm', 'avg_4_9_STL_pm', 'avg_10_19_STL_pm', 'avg_20_35_STL_pm', 'avg_36_85_STL_pm',
    'avg_2_3_BLK_pm', 'avg_4_9_BLK_pm', 'avg_10_19_BLK_pm', 'avg_20_35_BLK_pm', 'avg_36_85_BLK_pm',
    'avg_2_3_TO_pm', 'avg_4_9_TO_pm', 'avg_10_19_TO_pm', 'avg_20_35_TO_pm', 'avg_36_85_TO_pm',
    'avg_2_3_PTS_pm', 'avg_4_9_PTS_pm', 'avg_10_19_PTS_pm', 'avg_20_35_PTS_pm', 'avg_36_85_PTS_pm',
    'avg_2_3_PLUS_MINUS_pm', 'avg_4_9_PLUS_MINUS_pm', 'avg_10_19_PLUS_MINUS_pm', 'avg_20_35_PLUS_MINUS_pm', 'avg_36_85_PLUS_MINUS_pm',
    'avg_2_3_PF_pm', 'avg_4_9_PF_pm', 'avg_10_19_PF_pm', 'avg_20_35_PF_pm', 'avg_36_85_PF_pm',
    'avg_2_3_game_played', 'avg_4_9_game_played', 'avg_10_19_game_played', 'avg_20_35_game_played', 'avg_36_85_game_played'
    ]
created_back_to_back_window_cols = [
    'avg_1_20_played_in_first_game_of_back_to_back', 'avg_21_90_played_in_first_game_of_back_to_back',
    'avg_1_20_played_in_second_game_of_back_to_back', 'avg_21_90_played_in_second_game_of_back_to_back',
    ]
players_season_stats_cols = ['lag_1_MIN_season', 'lag_2_MIN_season', 'avg_3_22_MIN_season',
    'lag_1_game_played_season', 'lag_2_game_played_season', 'avg_3_22_game_played_season',
    'lag_1_FGM_pm_season', 'lag_2_FGM_pm_season', 'avg_3_22_FGM_pm_season',
    'lag_1_FGA_pm_season', 'lag_2_FGA_pm_season', 'avg_3_22_FGA_pm_season',
    'lag_1_FG3M_pm_season', 'lag_2_FG3M_pm_season', 'avg_3_22_FG3M_pm_season',
    'lag_1_FG3A_pm_season', 'lag_2_FG3A_pm_season', 'avg_3_22_FG3A_pm_season',
    'lag_1_FTM_pm_season', 'lag_2_FTM_pm_season', 'avg_3_22_FTM_pm_season',
    'lag_1_FTA_pm_season', 'lag_2_FTA_pm_season', 'avg_3_22_FTA_pm_season',
    'lag_1_REB_pm_season', 'lag_2_REB_pm_season', 'avg_3_22_REB_pm_season',
    'lag_1_AST_pm_season', 'lag_2_AST_pm_season', 'avg_3_22_AST_pm_season',
    'lag_1_STL_pm_season', 'lag_2_STL_pm_season', 'avg_3_22_STL_pm_season',
    'lag_1_BLK_pm_season', 'lag_2_BLK_pm_season', 'avg_3_22_BLK_pm_season',
    'lag_1_TO_pm_season', 'lag_2_TO_pm_season', 'avg_3_22_TO_pm_season',
    'lag_1_PTS_pm_season', 'lag_2_PTS_pm_season', 'avg_3_22_PTS_pm_season',
    'lag_1_PLUS_MINUS_pm_season', 'lag_2_PLUS_MINUS_pm_season', 'avg_3_22_PLUS_MINUS_pm_season',
    'lag_1_PF_pm_season', 'lag_2_PF_pm_season', 'avg_3_22_PF_pm_season',
    'lag_1_played_in_first_game_of_back_to_back_season', 'lag_2_played_in_first_game_of_back_to_back_season',
    'avg_3_22_played_in_first_game_of_back_to_back_season',
    'lag_1_played_in_second_game_of_back_to_back_season', 'lag_2_played_in_second_game_of_back_to_back_season',
    'avg_3_22_played_in_second_game_of_back_to_back_season'
    ]
matchup_forecast_cols = [
    'next_0_HOME', 'next_0_days_until_next_game', 'next_0_PTS_ALLOWED', 'next_0_FG_PCT_ALLOWED', 'next_0_FG3_PCT_ALLOWED', 'next_0_AST_ALLOWED', 'next_0_REB_ALLOWED',
    'next_1_HOME', 'next_1_days_until_next_game', 'next_1_PTS_ALLOWED', 'next_1_FG_PCT_ALLOWED', 'next_1_FG3_PCT_ALLOWED', 'next_1_AST_ALLOWED', 'next_1_REB_ALLOWED',
    'next_2_HOME', 'next_2_days_until_next_game', 'next_2_PTS_ALLOWED', 'next_2_FG_PCT_ALLOWED', 'next_2_FG3_PCT_ALLOWED', 'next_2_AST_ALLOWED', 'next_2_REB_ALLOWED',
    'next_3_HOME', 'next_3_days_until_next_game', 'next_3_PTS_ALLOWED', 'next_3_FG_PCT_ALLOWED', 'next_3_FG3_PCT_ALLOWED', 'next_3_AST_ALLOWED', 'next_3_REB_ALLOWED',
    'next_4_HOME', 'next_4_days_until_next_game', 'next_4_PTS_ALLOWED', 'next_4_FG_PCT_ALLOWED', 'next_4_FG3_PCT_ALLOWED', 'next_4_AST_ALLOWED', 'next_4_REB_ALLOWED',
]
other_cols = [
    'days_since_last_game', 'season_short', 'percentage_regular_season_games',
    ]
# print cols that are in the lists and not the df, and vice versa
lists_ = index_cols + all_labels + do_not_include + created_game_lag_cols \
    + created_window_lag_cols + created_window_avg_cols + created_back_to_back_window_cols \
    + players_season_stats_cols + matchup_forecast_cols + other_cols
add_cols = [col for col in df_start.columns if col not in lists_]
print(f'add col to lists ({len(add_cols)}):')
print(add_cols)
remove_cols = [col for col in lists_ if col not in df_start.columns]
print(f'remove col from lists ({len(remove_cols)}):')
print(remove_cols)


add col to lists (0):
[]
remove col from lists (0):
[]


In [33]:
df_start[all_labels] = df_start[all_labels].fillna(0)
df_start

,GAME_DATE_EST,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,season,days_since_last_game,season_short,percentage_regular_season_games,next_0_HOME,next_0_days_until_next_game,next_0_PTS_ALLOWED,next_0_FG_PCT_ALLOWED,next_0_FG3_PCT_ALLOWED,next_0_AST_ALLOWED,next_0_REB_ALLOWED,next_1_HOME,next_1_days_until_next_game,next_1_PTS_ALLOWED,next_1_FG_PCT_ALLOWED,next_1_FG3_PCT_ALLOWED,next_1_AST_ALLOWED,next_1_REB_ALLOWED,next_2_HOME,next_2_days_until_next_game,next_2_PTS_ALLOWED,next_2_FG_PCT_ALLOWED,next_2_FG3_PCT_ALLOWED,next_2_AST_ALLOWED,next_2_REB_ALLOWED,next_3_HOME,next_3_days_until_next_game,next_3_PTS_ALLOWED,next_3_FG_PCT_ALLOWED,next_3_FG3_PCT_ALLOWED,next_3_AST_ALLOWED,next_3_REB_ALLOWED,next_4_HOME,next_4_days_until_next_game,next_4_PTS_ALLOWED,next_4_FG_PCT_ALLOWED,next_4_FG3_PCT_ALLOWED,next_4_AST_ALLOWED,next_4_REB_ALLOWED,lag_1_START_POSITION,lag_1_comment_code,lag_1_comment_reason,lag_1_MIN,lag_1_FGM_pm,lag_1_FGA_pm,lag_1_FG3M_pm,lag_1_FG3A_pm,lag_1_FTM_pm,lag_1_FTA_pm,lag_1_REB_pm,lag_1_AST_pm,lag_1_STL_pm,lag_1_BLK_pm,lag_1_TO_pm,lag_1_PTS_pm,lag_1_PLUS_MINUS_pm,lag_1_PF_pm,lag_1_game_played,next_0_MIN,next_1_MIN,next_2_MIN,next_3_MIN,next_4_MIN,next_0_FGM,next_1_FGM,next_2_FGM,next_3_FGM,next_4_FGM,next_0_FGA,next_1_FGA,next_2_FGA,next_3_FGA,next_4_FGA,next_0_FG3M,next_1_FG3M,next_2_FG3M,next_3_FG3M,next_4_FG3M,next_0_FG3A,next_1_FG3A,next_2_FG3A,next_3_FG3A,next_4_FG3A,next_0_FTM,next_1_FTM,next_2_FTM,next_3_FTM,next_4_FTM,next_0_FTA,next_1_FTA,next_2_FTA,next_3_FTA,next_4_FTA,next_0_REB,next_1_REB,next_2_REB,next_3_REB,next_4_REB,next_0_AST,next_1_AST,next_2_AST,next_3_AST,next_4_AST,next_0_STL,next_1_STL,next_2_STL,next_3_STL,next_4_STL,next_0_BLK,next_1_BLK,next_2_BLK,next_3_BLK,next_4_BLK,next_0_TO,next_1_TO,next_2_TO,next_3_TO,next_4_TO,next_0_PTS,next_1_PTS,next_2_PTS,next_3_PTS,next_4_PTS,next_0_PLUS_MINUS,next_1_PLUS_MINUS,next_2_PLUS_MINUS,next_3_PLUS_MINUS,next_4_PLUS_MINUS,next_0_PF,next_1_PF,next_2_PF,next_3_PF,next_4_PF,avg_2_3_MIN,avg_4_9_MIN,avg_10_19_MIN,avg_20_35_MIN,avg_36_85_MIN,avg_2_3_FGM_pm,avg_4_9_FGM_pm,avg_10_19_FGM_pm,avg_20_35_FGM_pm,avg_36_85_FGM_pm,avg_2_3_FGA_pm,avg_4_9_FGA_pm,avg_10_19_FGA_pm,avg_20_35_FGA_pm,avg_36_85_FGA_pm,avg_2_3_FG3M_pm,avg_4_9_FG3M_pm,avg_10_19_FG3M_pm,avg_20_35_FG3M_pm,avg_36_85_FG3M_pm,avg_2_3_FG3A_pm,avg_4_9_FG3A_pm,avg_10_19_FG3A_pm,avg_20_35_FG3A_pm,avg_36_85_FG3A_pm,avg_2_3_FTM_pm,avg_4_9_FTM_pm,avg_10_19_FTM_pm,avg_20_35_FTM_pm,avg_36_85_FTM_pm,avg_2_3_FTA_pm,avg_4_9_FTA_pm,avg_10_19_FTA_pm,avg_20_35_FTA_pm,avg_36_85_FTA_pm,avg_2_3_REB_pm,avg_4_9_REB_pm,avg_10_19_REB_pm,avg_20_35_REB_pm,avg_36_85_REB_pm,avg_2_3_AST_pm,avg_4_9_AST_pm,avg_10_19_AST_pm,avg_20_35_AST_pm,avg_36_85_AST_pm,avg_2_3_STL_pm,avg_4_9_STL_pm,avg_10_19_STL_pm,avg_20_35_STL_pm,avg_36_85_STL_pm,avg_2_3_BLK_pm,avg_4_9_BLK_pm,avg_10_19_BLK_pm,avg_20_35_BLK_pm,avg_36_85_BLK_pm,avg_2_3_TO_pm,avg_4_9_TO_pm,avg_10_19_TO_pm,avg_20_35_TO_pm,avg_36_85_TO_pm,avg_2_3_PTS_pm,avg_4_9_PTS_pm,avg_10_19_PTS_pm,avg_20_35_PTS_pm,avg_36_85_PTS_pm,avg_2_3_PLUS_MINUS_pm,avg_4_9_PLUS_MINUS_pm,avg_10_19_PLUS_MINUS_pm,avg_20_35_PLUS_MINUS_pm,avg_36_85_PLUS_MINUS_pm,avg_2_3_PF_pm,avg_4_9_PF_pm,avg_10_19_PF_pm,avg_20_35_PF_pm,avg_36_85_PF_pm,avg_2_3_game_played,avg_4_9_game_played,avg_10_19_game_played,avg_20_35_game_played,avg_36_85_game_played,avg_1_20_played_in_first_game_of_back_to_back,avg_21_90_played_in_first_game_of_back_to_back,avg_1_20_played_in_second_game_of_back_to_back,avg_21_90_played_in_second_game_of_back_to_back,lag_1_MIN_season,lag_2_MIN_season,lag_1_game_played_season,lag_2_game_played_season,lag_1_FGM_pm_season,lag_2_FGM_pm_season,lag_1_FGA_pm_season,lag_2_FGA_pm_season,lag_1_FG3M_pm_season,lag_2_FG3M_pm_season,lag_1_FG3A_pm_season,lag_2_FG3A_pm_season,lag_1_FTM_pm_season,lag_2_FTM_pm_season,lag_1_FTA_pm_season,lag_2_FTA_pm_season,lag_1_REB_pm_season,lag_2_REB_pm_season,lag_1_AST_pm_season,lag_2_AST_pm_season,lag_1_STL_pm_season,lag_2_STL_pm_season,lag_1_BLK_pm_season,lag_2_BLK_pm_season,lag_1_TO_

In [34]:
cat_features = created_game_lag_cols
num_features = created_window_lag_cols + created_window_avg_cols + created_back_to_back_window_cols \
        + players_season_stats_cols + matchup_forecast_cols + other_cols
features = cat_features + num_features

In [35]:
## TODO
## - Select columns to keep
var_dict_ts = {'num_vars': num_features + all_labels,
    'cat_vars': cat_features,
}

## - Explore data
explore_ts = ExploreData(df_start, var_dict=var_dict_ts)
explore_ts.summary()
# explore_ts.target_variable(target='MIN')

========= Summary Statistics =========
579662 rows, 276 columns

========= Column Level Statistics =========
GAME_DATE_EST -- object
Column not in cat_vars or num_vars
Unique values: 3109
Null count: 0

GAME_ID -- int64
Column not in cat_vars or num_vars
Unique values: 23290
Null count: 0

TEAM_ID -- int64
Column not in cat_vars or num_vars
Unique values: 30
Null count: 0

PLAYER_ID -- int64
Column not in cat_vars or num_vars
Unique values: 2094
Null count: 0

PLAYER_NAME -- object
Column not in cat_vars or num_vars
Unique values: 2093
Null count: 0

season -- object
Column not in cat_vars or num_vars
Unique values: 20
Null count: 0

days_since_last_game -- int64
count    579662.000000
mean          2.209538
std           3.445814
min           1.000000
1%            1.000000
10%           1.000000
25%           2.000000
50%           2.000000
75%           2.000000
90%           3.000000
99%           7.000000
max         138.000000
Name: days_since_last_game, dtype: float64
number of

In [36]:
labels = all_labels.copy()
# labels = ['next_0_MIN', 'next_1_MIN', 'next_2_MIN', 'next_3_MIN', 'next_4_MIN', 'next_0_PTS', 'next_1_PTS', 'next_2_PTS', 'next_3_PTS', 'next_4_PTS']
df_ts = df_start[index_cols + labels + features].copy()
df_ts

,GAME_DATE_EST,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,season,next_0_MIN,next_1_MIN,next_2_MIN,next_3_MIN,next_4_MIN,next_0_PTS,next_1_PTS,next_2_PTS,next_3_PTS,next_4_PTS,next_0_FGM,next_1_FGM,next_2_FGM,next_3_FGM,next_4_FGM,next_0_FGA,next_1_FGA,next_2_FGA,next_3_FGA,next_4_FGA,next_0_FTM,next_1_FTM,next_2_FTM,next_3_FTM,next_4_FTM,next_0_FTA,next_1_FTA,next_2_FTA,next_3_FTA,next_4_FTA,next_0_REB,next_1_REB,next_2_REB,next_3_REB,next_4_REB,next_0_AST,next_1_AST,next_2_AST,next_3_AST,next_4_AST,next_0_STL,next_1_STL,next_2_STL,next_3_STL,next_4_STL,next_0_BLK,next_1_BLK,next_2_BLK,next_3_BLK,next_4_BLK,next_0_TO,next_1_TO,next_2_TO,next_3_TO,next_4_TO,lag_1_START_POSITION,lag_1_comment_code,lag_1_comment_reason,lag_1_MIN,lag_1_FGM_pm,lag_1_FGA_pm,lag_1_FG3M_pm,lag_1_FG3A_pm,lag_1_FTM_pm,lag_1_FTA_pm,lag_1_REB_pm,lag_1_AST_pm,lag_1_STL_pm,lag_1_BLK_pm,lag_1_TO_pm,lag_1_PTS_pm,lag_1_PLUS_MINUS_pm,lag_1_PF_pm,lag_1_game_played,avg_2_3_MIN,avg_4_9_MIN,avg_10_19_MIN,avg_20_35_MIN,avg_36_85_MIN,avg_2_3_FGM_pm,avg_4_9_FGM_pm,avg_10_19_FGM_pm,avg_20_35_FGM_pm,avg_36_85_FGM_pm,avg_2_3_FGA_pm,avg_4_9_FGA_pm,avg_10_19_FGA_pm,avg_20_35_FGA_pm,avg_36_85_FGA_pm,avg_2_3_FG3M_pm,avg_4_9_FG3M_pm,avg_10_19_FG3M_pm,avg_20_35_FG3M_pm,avg_36_85_FG3M_pm,avg_2_3_FG3A_pm,avg_4_9_FG3A_pm,avg_10_19_FG3A_pm,avg_20_35_FG3A_pm,avg_36_85_FG3A_pm,avg_2_3_FTM_pm,avg_4_9_FTM_pm,avg_10_19_FTM_pm,avg_20_35_FTM_pm,avg_36_85_FTM_pm,avg_2_3_FTA_pm,avg_4_9_FTA_pm,avg_10_19_FTA_pm,avg_20_35_FTA_pm,avg_36_85_FTA_pm,avg_2_3_REB_pm,avg_4_9_REB_pm,avg_10_19_REB_pm,avg_20_35_REB_pm,avg_36_85_REB_pm,avg_2_3_AST_pm,avg_4_9_AST_pm,avg_10_19_AST_pm,avg_20_35_AST_pm,avg_36_85_AST_pm,avg_2_3_STL_pm,avg_4_9_STL_pm,avg_10_19_STL_pm,avg_20_35_STL_pm,avg_36_85_STL_pm,avg_2_3_BLK_pm,avg_4_9_BLK_pm,avg_10_19_BLK_pm,avg_20_35_BLK_pm,avg_36_85_BLK_pm,avg_2_3_TO_pm,avg_4_9_TO_pm,avg_10_19_TO_pm,avg_20_35_TO_pm,avg_36_85_TO_pm,avg_2_3_PTS_pm,avg_4_9_PTS_pm,avg_10_19_PTS_pm,avg_20_35_PTS_pm,avg_36_85_PTS_pm,avg_2_3_PLUS_MINUS_pm,avg_4_9_PLUS_MINUS_pm,avg_10_19_PLUS_MINUS_pm,avg_20_35_PLUS_MINUS_pm,avg_36_85_PLUS_MINUS_pm,avg_2_3_PF_pm,avg_4_9_PF_pm,avg_10_19_PF_pm,avg_20_35_PF_pm,avg_36_85_PF_pm,avg_2_3_game_played,avg_4_9_game_played,avg_10_19_game_played,avg_20_35_game_played,avg_36_85_game_played,avg_1_20_played_in_first_game_of_back_to_back,avg_21_90_played_in_first_game_of_back_to_back,avg_1_20_played_in_second_game_of_back_to_back,avg_21_90_played_in_second_game_of_back_to_back,lag_1_MIN_season,lag_2_MIN_season,avg_3_22_MIN_season,lag_1_game_played_season,lag_2_game_played_season,avg_3_22_game_played_season,lag_1_FGM_pm_season,lag_2_FGM_pm_season,avg_3_22_FGM_pm_season,lag_1_FGA_pm_season,lag_2_FGA_pm_season,avg_3_22_FGA_pm_season,lag_1_FG3M_pm_season,lag_2_FG3M_pm_season,avg_3_22_FG3M_pm_season,lag_1_FG3A_pm_season,lag_2_FG3A_pm_season,avg_3_22_FG3A_pm_season,lag_1_FTM_pm_season,lag_2_FTM_pm_season,avg_3_22_FTM_pm_season,lag_1_FTA_pm_season,lag_2_FTA_pm_season,avg_3_22_FTA_pm_season,lag_1_REB_pm_season,lag_2_REB_pm_season,avg_3_22_REB_pm_season,lag_1_AST_pm_season,lag_2_AST_pm_season,avg_3_22_AST_pm_season,lag_1_STL_pm_season,lag_2_STL_pm_season,avg_3_22_STL_pm_season,lag_1_BLK_pm_season,lag_2_BLK_pm_season,avg_3_22_BLK_pm_season,lag_1_TO_pm_season,lag_2_TO_pm_season,avg_3_22_TO_pm_season,lag_1_PTS_pm_season,lag_2_PTS_pm_season,avg_3_22_PTS_pm_season,lag_1_PLUS_MINUS_pm_season,lag_2_PLUS_MINUS_pm_season,avg_3_22_PLUS_MINUS_pm_season,lag_1_PF_pm_season,lag_2_PF_pm_season,avg_3_22_PF_pm_season,lag_1_played_in_first_game_of_back_to_back_season,lag_2_played_in_first_game_of_back_to_back_season,avg_3_22_played_in_first_game_of_back_to_back_season,lag_1_played_in_second_game_of_back_to_back_season,lag_2_played_in_second_game_of_back_to_back_season,avg_3_22_played_in_second_game_of_back_to_back_season,next_0_HOME,next_0_days_until_next_game,next_0_PTS_ALLOWED,next_0_FG_PCT_ALLOWED,next_0_FG3_PCT_ALLOWED,next_0_AST_ALLOWED,next_0_REB_ALLOWED,next_1_HOME,next_1_days_until_next_game,next_1_

In [37]:
df_ts['season_short']

0         2004
1         2004
2         2004
3         2004
4         2004
          ... 
579657    2023
579658    2023
579659    2023
579660    2023
579661    2023
Name: season_short, Length: 579662, dtype: int64

In [38]:
# train data is season_short 2007-2018 and 2021, validation data is 2019 and 2022, test data is 2020 and 2023
train_mask = (df_ts['season_short'].between(2007, 2018)) | (df_ts['season_short'] == 2021)
val_mask = df_ts['season_short'].between(2019, 2020)
train_val_mask = train_mask | val_mask
test_mask = df_ts['season_short'].between(2022, 2023)
train_data = df_ts.loc[train_mask].reset_index()
val_data = df_ts.loc[val_mask].reset_index()
train_val_data = df_ts.loc[train_val_mask].reset_index()
test_data = df_ts.loc[test_mask].reset_index()

## seperate each of train, val, and test into their own X and y
X_train = train_data[features].copy()
y_train = train_data[labels].copy()
idx_train = train_data[index_cols].copy()
X_val = val_data[features].copy()
y_val = val_data[labels].copy()
idx_val = val_data[index_cols].copy()
X_train_val = train_val_data[features].copy()
y_train_val = train_val_data[labels].copy()
idx_train_val = train_val_data[index_cols].copy()
X_test = test_data[features].copy()
y_test = test_data[labels].copy()
idx_test = test_data[index_cols].copy()

train_features = X_train.columns.tolist()

print('train features shape:', X_train.shape)
print('train labels shape:', y_train.shape)
print('train index columns shape:', y_train.shape)
print('val data shape:', val_data.shape)
print('test data shape:', test_data.shape)

train features shape: (390303, 195)
train labels shape: (390303, 55)
train index columns shape: (390303, 55)
val data shape: (58410, 257)
test data shape: (43667, 257)


In [39]:
## num_features_missing got from running the model with all missing features added then 
## looking at feature importance to see which features had a positive feature importance.
## If a feature never had importance for any label, then it was removed from the list.
num_features_to_add_missing_indicator = [
    'lag_1_MIN', 'lag_1_FGM_pm', 'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN',
    'avg_2_3_FGM_pm', 'avg_4_9_FGM_pm', 'avg_10_19_FGM_pm', 'avg_20_35_FGM_pm', 'avg_36_85_FGM_pm',
    'avg_1_20_played_in_first_game_of_back_to_back', 'avg_21_90_played_in_first_game_of_back_to_back',
    'avg_1_20_played_in_second_game_of_back_to_back', 'avg_21_90_played_in_second_game_of_back_to_back',
    'lag_1_MIN_season', 'lag_2_MIN_season', 'avg_3_22_MIN_season', 'lag_1_FGM_pm_season', 'lag_2_FGM_pm_season',
    'lag_1_played_in_first_game_of_back_to_back_season', 'lag_2_played_in_first_game_of_back_to_back_season',
    'lag_1_played_in_second_game_of_back_to_back_season', 'lag_2_played_in_second_game_of_back_to_back_season',
    'next_0_HOME', 'next_1_HOME', 'next_2_HOME', 'next_3_HOME', 'next_4_HOME']

num_features_missing = [col for col in num_features if train_data[col].isnull().sum() > 0]
cat_features_missing = [col for col in cat_features if train_data[col].isnull().sum() > 0]
print(f'num_features_missing ({len(num_features_missing)}): {num_features_missing}')
print(f'cat_features_missing ({len(cat_features_missing)}): {cat_features_missing}')

num_features_missing (189): ['lag_1_MIN', 'lag_1_FGM_pm', 'lag_1_FGA_pm', 'lag_1_FG3M_pm', 'lag_1_FG3A_pm', 'lag_1_FTM_pm', 'lag_1_FTA_pm', 'lag_1_REB_pm', 'lag_1_AST_pm', 'lag_1_STL_pm', 'lag_1_BLK_pm', 'lag_1_TO_pm', 'lag_1_PTS_pm', 'lag_1_PLUS_MINUS_pm', 'lag_1_PF_pm', 'lag_1_game_played', 'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN', 'avg_2_3_FGM_pm', 'avg_4_9_FGM_pm', 'avg_10_19_FGM_pm', 'avg_20_35_FGM_pm', 'avg_36_85_FGM_pm', 'avg_2_3_FGA_pm', 'avg_4_9_FGA_pm', 'avg_10_19_FGA_pm', 'avg_20_35_FGA_pm', 'avg_36_85_FGA_pm', 'avg_2_3_FG3M_pm', 'avg_4_9_FG3M_pm', 'avg_10_19_FG3M_pm', 'avg_20_35_FG3M_pm', 'avg_36_85_FG3M_pm', 'avg_2_3_FG3A_pm', 'avg_4_9_FG3A_pm', 'avg_10_19_FG3A_pm', 'avg_20_35_FG3A_pm', 'avg_36_85_FG3A_pm', 'avg_2_3_FTM_pm', 'avg_4_9_FTM_pm', 'avg_10_19_FTM_pm', 'avg_20_35_FTM_pm', 'avg_36_85_FTM_pm', 'avg_2_3_FTA_pm', 'avg_4_9_FTA_pm', 'avg_10_19_FTA_pm', 'avg_20_35_FTA_pm', 'avg_36_85_FTA_pm', 'avg_2_3_REB_pm', 'avg_4_9_REB_pm', 'av

In [40]:
from feature_engine.imputation import CategoricalImputer, AddMissingIndicator, MeanMedianImputer
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

print('Imputing categorical variables with missing data...')
cat_imputer = CategoricalImputer(
    imputation_method="missing",
    variables=cat_features_missing,
)
cat_imputer.fit(X_train)
X_train = cat_imputer.transform(X_train)
X_val = cat_imputer.transform(X_val)
X_train_val = cat_imputer.transform(X_train_val)
X_test = cat_imputer.transform(X_test)

print('Adding missing indicator for categorical variables...')
num_missing_indicator = AddMissingIndicator(variables=num_features_to_add_missing_indicator)
num_missing_indicator.fit(X_train)
X_train = num_missing_indicator.transform(X_train)
X_val = num_missing_indicator.transform(X_val)
X_train_val = num_missing_indicator.transform(X_train_val)
X_test = num_missing_indicator.transform(X_test)

print('Imputing numerical variables with mean...')
meanimputer = MeanMedianImputer(
    imputation_method="mean",
    variables=num_features_missing,
)
meanimputer.fit(X_train)
X_train = meanimputer.transform(X_train)
X_val = meanimputer.transform(X_val)
X_train_val = meanimputer.transform(X_train_val)
X_test = meanimputer.transform(X_test)

print('Encoding rare labels in categorical variables...')
encoder = RareLabelEncoder(
    tol=0.01, n_categories=1, variables=cat_features, ignore_format=True
)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_val = encoder.transform(X_val)
X_train_val = encoder.transform(X_train_val)
X_test = encoder.transform(X_test)

## NOTE: new categories are automatically ignored
print('One hot encoding categorical variables...')
ohe = OneHotEncoder(
    drop_last_binary=True,
    variables=cat_features, ignore_format=True
)
ohe.fit(X_train)
X_train = ohe.transform(X_train)
X_val = ohe.transform(X_val)
X_train_val = ohe.transform(X_train_val)
X_test = ohe.transform(X_test)

col_names = X_train.columns.tolist()

print('Scaling numerical variables...')
scaler = MinMaxScaler()  # do you need a scaler with randomforest and lightgbm? -- no
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=col_names)
X_val = pd.DataFrame(scaler.transform(X_val), columns=col_names)
X_train_val = pd.DataFrame(scaler.transform(X_train_val), columns=col_names)
X_test = pd.DataFrame(scaler.transform(X_test), columns=col_names)

print('train features shape:', X_train.shape)
print('train labels shape:', y_train.shape)
display(X_train.head())

Imputing categorical variables with missing data...
Adding missing indicator for categorical variables...
Imputing numerical variables with mean...
Encoding rare labels in categorical variables...
One hot encoding categorical variables...
Scaling numerical variables...
train features shape: (390303, 233)
train labels shape: (390303, 55)


,lag_1_MIN,lag_1_FGM_pm,lag_1_FGA_pm,lag_1_FG3M_pm,lag_1_FG3A_pm,lag_1_FTM_pm,lag_1_FTA_pm,lag_1_REB_pm,lag_1_AST_pm,lag_1_STL_pm,lag_1_BLK_pm,lag_1_TO_pm,lag_1_PTS_pm,lag_1_PLUS_MINUS_pm,lag_1_PF_pm,lag_1_game_played,avg_2_3_MIN,avg_4_9_MIN,avg_10_19_MIN,avg_20_35_MIN,avg_36_85_MIN,avg_2_3_FGM_pm,avg_4_9_FGM_pm,avg_10_19_FGM_pm,avg_20_35_FGM_pm,avg_36_85_FGM_pm,avg_2_3_FGA_pm,avg_4_9_FGA_pm,avg_10_19_FGA_pm,avg_20_35_FGA_pm,avg_36_85_FGA_pm,avg_2_3_FG3M_pm,avg_4_9_FG3M_pm,avg_10_19_FG3M_pm,avg_20_35_FG3M_pm,avg_36_85_FG3M_pm,avg_2_3_FG3A_pm,avg_4_9_FG3A_pm,avg_10_19_FG3A_pm,avg_20_35_FG3A_pm,avg_36_85_FG3A_pm,avg_2_3_FTM_pm,avg_4_9_FTM_pm,avg_10_19_FTM_pm,avg_20_35_FTM_pm,avg_36_85_FTM_pm,avg_2_3_FTA_pm,avg_4_9_FTA_pm,avg_10_19_FTA_pm,avg_20_35_FTA_pm,avg_36_85_FTA_pm,avg_2_3_REB_pm,avg_4_9_REB_pm,avg_10_19_REB_pm,avg_20_35_REB_pm,avg_36_85_REB_pm,avg_2_3_AST_pm,avg_4_9_AST_pm,avg_10_19_AST_pm,avg_20_35_AST_pm,avg_36_85_AST_pm,avg_2_3_STL_pm,avg_4_9_STL_pm,avg_10_19_STL_pm,avg_20_35_STL_pm,avg_36_85_STL_pm,avg_2_3_BLK_pm,avg_4_9_BLK_pm,avg_10_19_BLK_pm,avg_20_35_BLK_pm,avg_36_85_BLK_pm,avg_2_3_TO_pm,avg_4_9_TO_pm,avg_10_19_TO_pm,avg_20_35_TO_pm,avg_36_85_TO_pm,avg_2_3_PTS_pm,avg_4_9_PTS_pm,avg_10_19_PTS_pm,avg_20_35_PTS_pm,avg_36_85_PTS_pm,avg_2_3_PLUS_MINUS_pm,avg_4_9_PLUS_MINUS_pm,avg_10_19_PLUS_MINUS_pm,avg_20_35_PLUS_MINUS_pm,avg_36_85_PLUS_MINUS_pm,avg_2_3_PF_pm,avg_4_9_PF_pm,avg_10_19_PF_pm,avg_20_35_PF_pm,avg_36_85_PF_pm,avg_2_3_game_played,avg_4_9_game_played,avg_10_19_game_played,avg_20_35_game_played,avg_36_85_game_played,avg_1_20_played_in_first_game_of_back_to_back,avg_21_90_played_in_first_game_of_back_to_back,avg_1_20_played_in_second_game_of_back_to_back,avg_21_90_played_in_second_game_of_back_to_back,lag_1_MIN_season,lag_2_MIN_season,avg_3_22_MIN_season,lag_1_game_played_season,lag_2_game_played_season,avg_3_22_game_played_season,lag_1_FGM_pm_season,lag_2_FGM_pm_season,avg_3_22_FGM_pm_season,lag_1_FGA_pm_season,lag_2_FGA_pm_season,avg_3_22_FGA_pm_season,lag_1_FG3M_pm_season,lag_2_FG3M_pm_season,avg_3_22_FG3M_pm_season,lag_1_FG3A_pm_season,lag_2_FG3A_pm_season,avg_3_22_FG3A_pm_season,lag_1_FTM_pm_season,lag_2_FTM_pm_season,avg_3_22_FTM_pm_season,lag_1_FTA_pm_season,lag_2_FTA_pm_season,avg_3_22_FTA_pm_season,lag_1_REB_pm_season,lag_2_REB_pm_season,avg_3_22_REB_pm_season,lag_1_AST_pm_season,lag_2_AST_pm_season,avg_3_22_AST_pm_season,lag_1_STL_pm_season,lag_2_STL_pm_season,avg_3_22_STL_pm_season,lag_1_BLK_pm_season,lag_2_BLK_pm_season,avg_3_22_BLK_pm_season,lag_1_TO_pm_season,lag_2_TO_pm_season,avg_3_22_TO_pm_season,lag_1_PTS_pm_season,lag_2_PTS_pm_season,avg_3_22_PTS_pm_season,lag_1_PLUS_MINUS_pm_season,lag_2_PLUS_MINUS_pm_season,avg_3_22_PLUS_MINUS_pm_season,lag_1_PF_pm_season,lag_2_PF_pm_season,avg_3_22_PF_pm_season,lag_1_played_in_first_game_of_back_to_back_season,lag_2_played_in_first_game_of_back_to_back_season,avg_3_22_played_in_first_game_of_back_to_back_season,lag_1_played_in_second_game_of_back_to_back_season,lag_2_played_in_second_game_of_back_to_back_season,avg_3_22_played_in_second_game_of_back_to_back_season,next_0_HOME,next_0_days_until_next_game,next_0_PTS_ALLOWED,next_0_FG_PCT_ALLOWED,next_0_FG3_PCT_ALLOWED,next_0_AST_ALLOWED,next_0_REB_ALLOWED,next_1_HOME,next_1_days_until_next_game,next_1_PTS_ALLOWED,next_1_FG_PCT_ALLOWED,next_1_FG3_PCT_ALLOWED,next_1_AST_ALLOWED,next_1_REB_ALLOWED,next_2_HOME,next_2_days_until_next_game,next_2_PTS_ALLOWED,next_2_FG_PCT_ALLOWED,next_2_FG3_PCT_ALLOWED,next_2_AST_ALLOWED,next_2_REB_ALLOWED,next_3_HOME,next_3_days_until_next_game,next_3_PTS_ALLOWED,next_3_FG_PCT_ALLOWED,next_3_FG3_PCT_ALLOWED,next_3_AST_ALLOWED,next_3_REB_ALLOWED,next_4_HOME,next_4_days_until_next_game,next_4_PTS_ALLOWED,next_4_FG_PCT_ALLOWED,next_4_FG3_PCT_ALLOWED,next_4_AST_ALLOWED,next_4_REB_ALLOWED,days_since_last_game,season_short,percentage_regular_season_games,lag_1_MIN_na,lag_1_FGM_pm_na,avg_2_3_MIN_na,avg_4_9_MIN_na,avg_10_19_MIN_na,avg_20_35_MIN_na,avg_36_85_MIN_na,avg_2_3_FGM_pm_na,avg_4

In [41]:
# ## Train RegressorChain LightGBM model and dump it to a joblib file
# import joblib
# from sklearn.multioutput import RegressorChain
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_squared_error
# from math import sqrt

# # define base model
# model = LGBMRegressor()
# # define the chained multioutput wrapper model
# chain = RegressorChain(model)
# chain.fit(X_train_val, y_train_val)
# yhat = chain.predict(X_test)

# mse = mean_squared_error(y_test, yhat)
# rmse = sqrt(mse)
# print('RMSE: %f' % rmse)

# joblib.dump(chain, 'chain_lightGBM.joblib')

In [42]:
import joblib
from sklearn.metrics import mean_squared_error
from math import sqrt

chain2 = joblib.load('chain_lightGBM.joblib')
yhat2 = chain2.predict(X_test)
mse2 = mean_squared_error(y_test, yhat2)
rmse2 = sqrt(mse2)
print('RMSE: %f' % rmse2)

RMSE: 3.748062


In [43]:
y_sample1 =y_test.reset_index().iloc[[1, 39789]]
X_sample1 = X_test.iloc[[1, 39789]]
yhat_sample1 = chain2.predict(X_sample1)
yhat_sample1 = pd.DataFrame(yhat_sample1, columns=labels)
yhat_sample1 = yhat_sample1.set_index(y_sample1.index)

display(yhat_sample1)
display(y_sample1)

,next_0_MIN,next_1_MIN,next_2_MIN,next_3_MIN,next_4_MIN,next_0_PTS,next_1_PTS,next_2_PTS,next_3_PTS,next_4_PTS,next_0_FGM,next_1_FGM,next_2_FGM,next_3_FGM,next_4_FGM,next_0_FGA,next_1_FGA,next_2_FGA,next_3_FGA,next_4_FGA,next_0_FTM,next_1_FTM,next_2_FTM,next_3_FTM,next_4_FTM,next_0_FTA,next_1_FTA,next_2_FTA,next_3_FTA,next_4_FTA,next_0_REB,next_1_REB,next_2_REB,next_3_REB,next_4_REB,next_0_AST,next_1_AST,next_2_AST,next_3_AST,next_4_AST,next_0_STL,next_1_STL,next_2_STL,next_3_STL,next_4_STL,next_0_BLK,next_1_BLK,next_2_BLK,next_3_BLK,next_4_BLK,next_0_TO,next_1_TO,next_2_TO,next_3_TO,next_4_TO
1,28.120554,28.095127,27.470596,27.209638,26.165147,8.385075,8.400374,7.894734,7.545381,6.962634,3.114379,3.099848,3.112863,3.067343,2.640008,6.455608,6.285723,6.244515,6.280066,6.156860,1.153262,0.920627,1.075853,1.069393,0.390717,1.914168,1.854668,1.874520,1.815599,1.825186,5.977907,5.979100,5.729708,5.718566,5.520575,6.411234,6.540413,6.297006,6.457686,5.743552,1.478077,1.513873,1.388526,1.441042,1.298388,0.624212,0.664449,0.617716,0.70198,0.646995,2.299048,2.523814,2.505379,2.656875,2.475024
39789,19.482739,18.433788,18.443730,17.838473,17.555853,8.403793,8.021585,8.037615,7.680658,7.974567,3.005722,2.966017,2.948053,2.979063,2.988381,7.593316,7.236273,7.207597,7.055542,7.286956,0.481412,0.576073,0.633147,0.581796,0.563317,1.667073,1.637102,1.645705,1.662058,1.668964,2.225694,2.082197,1.932590,1.904085,1.874278,2.137028,2.220446,2.104242,2.012859,2.066729,0.712042,0.713286,0.695168,0.690085,0.700435,0.269006,0.267150,0.312062,0.32675,0.381630,1.126609,1.195687,1.102591,1.045004,1.041774


,index,next_0_MIN,next_1_MIN,next_2_MIN,next_3_MIN,next_4_MIN,next_0_PTS,next_1_PTS,next_2_PTS,next_3_PTS,next_4_PTS,next_0_FGM,next_1_FGM,next_2_FGM,next_3_FGM,next_4_FGM,next_0_FGA,next_1_FGA,next_2_FGA,next_3_FGA,next_4_FGA,next_0_FTM,next_1_FTM,next_2_FTM,next_3_FTM,next_4_FTM,next_0_FTA,next_1_FTA,next_2_FTA,next_3_FTA,next_4_FTA,next_0_REB,next_1_REB,next_2_REB,next_3_REB,next_4_REB,next_0_AST,next_1_AST,next_2_AST,next_3_AST,next_4_AST,next_0_STL,next_1_STL,next_2_STL,next_3_STL,next_4_STL,next_0_BLK,next_1_BLK,next_2_BLK,next_3_BLK,next_4_BLK,next_0_TO,next_1_TO,next_2_TO,next_3_TO,next_4_TO
1,1,29.933333,33.716667,34.383333,32.083333,37.616667,6.0,10.0,14.0,8.0,4.0,2.0,4.0,6.0,3.0,2.0,5.0,5.0,12.0,4.0,5.0,2.0,2.0,1.0,1.0,0.0,4.0,9.0,2.0,2.0,0.0,8.0,6.0,6.0,9.0,12.0,6.0,7.0,6.0,8.0,7.0,1.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0,1.0,5.0,5.0,1.0,1.0,4.0,5.0
39789,39789,15.466667,3.483333,0.000000,28.883333,25.133333,11.0,0.0,0.0,11.0,11.0,5.0,0.0,0.0,4.0,4.0,7.0,1.0,0.0,5.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,7.0,4.0,0.0,2.0,0.0,4.0,4.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0


In [44]:
feat_imps = {}

for i, label in enumerate(labels):
    feat_imps[label] = chain2.estimators_[i].feature_importances_[:-i] if i > 0 else chain2.estimators_[i].feature_importances_

# feat_imps
feat_imps = pd.DataFrame(feat_imps, index=X_train.columns)
feats_w_zero_importance = feat_imps.max(axis=1)[feat_imps.max(axis=1) == 0].index.tolist()
feats_w_importance = feat_imps.max(axis=1)[feat_imps.max(axis=1) > 0].index.tolist()

non_na_feats_w_zero_importance = [x for x in feats_w_zero_importance if not x.endswith('_na')]
print('non NaN features that have 0 importance:', non_na_feats_w_zero_importance)

feats_to_remove = [x for x in feats_w_zero_importance if x not in non_na_feats_w_zero_importance]
print('NaN features to remove:', len(feats_to_remove))

na_features_w_importance = [x for x in feats_w_importance if x.endswith('_na')]
print('number of NaN features that have >0 importance:', len(na_features_w_importance))
print(na_features_w_importance)

# for label, feat_imp in :
#     print(label, feat_imp)

# label_dict = {label:values for label,values in zip(feat_imps.keys(), list(zip(*feat_imps.values())))}

# for label, values in label_dict.items():
#     print(f"{label} - {max(values)} - {min(values)}")
# len(labels)

non NaN features that have 0 importance: []
NaN features to remove: 0
number of NaN features that have >0 importance: 30
['lag_1_MIN_na', 'lag_1_FGM_pm_na', 'avg_2_3_MIN_na', 'avg_4_9_MIN_na', 'avg_10_19_MIN_na', 'avg_20_35_MIN_na', 'avg_36_85_MIN_na', 'avg_2_3_FGM_pm_na', 'avg_4_9_FGM_pm_na', 'avg_10_19_FGM_pm_na', 'avg_20_35_FGM_pm_na', 'avg_36_85_FGM_pm_na', 'avg_1_20_played_in_first_game_of_back_to_back_na', 'avg_21_90_played_in_first_game_of_back_to_back_na', 'avg_1_20_played_in_second_game_of_back_to_back_na', 'avg_21_90_played_in_second_game_of_back_to_back_na', 'lag_1_MIN_season_na', 'lag_2_MIN_season_na', 'avg_3_22_MIN_season_na', 'lag_1_FGM_pm_season_na', 'lag_2_FGM_pm_season_na', 'lag_1_played_in_first_game_of_back_to_back_season_na', 'lag_2_played_in_first_game_of_back_to_back_season_na', 'lag_1_played_in_second_game_of_back_to_back_season_na', 'lag_2_played_in_second_game_of_back_to_back_season_na', 'next_0_HOME_na', 'next_1_HOME_na', 'next_2_HOME_na', 'next_3_HOME_na', 'n

In [45]:
## write X_test, y_test, idx_test to csv files

X_test.to_csv(ml_path + 'X_test.csv', index=False)
y_test.to_csv(ml_path + 'y_test.csv', index=False)
idx_test.to_csv(ml_path + 'idx_test.csv', index=False)

In [ ]:
## TODO
## - -- DONE -- Train model
## - -- DONE -- make predictions
## - -- DONE -- choose a season and a few Player IDs that would be good for examples
##    - injuries, trades, low/high minutes, low/high usage, rookie
## - create a function to chart predictions vs reality
## - do the chart

In [ ]:
## TODO:
## If we are going Boruta ---
    ## Update boruta script to write to csv after each label
    ## Create test boruta dataset that can be run faster to test writing to csv
    ## Run Boruta for each label (future games probably not needed)
## If we are not going Boruta ---
    ## Test each label using SelectFromModel
## Select features -- any feautre that gets a 1 or 2 at least once is selected
## Build model with selected features

In [35]:
# from boruta import BorutaPy
# from sklearn.ensemble import RandomForestRegressor

# # instantiate random forest
# forest = RandomForestRegressor(n_jobs = -1, max_depth = 5)

# # fit boruta
# boruta_selector = BorutaPy(forest, random_state = 0, n_estimators = 10) # 'auto')
# boruta_selector.fit(np.array(X_train_min), np.array(y_train_min).reshape(-1))

# # store results
# boruta_ranking = boruta_selector.ranking_
# selected_features = np.array(col_names)[boruta_ranking <= 2]
# selected_features

array(['percentage_regular_season_games', 'lag_1_MIN', 'avg_4_9_WIN',
       'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_20_35_MIN',
       'avg_2_3_game_played', 'lag_1_MIN_season',
       'lag_1_played_in_first_game_of_back_to_back_season',
       'lag_1_MIN_na', 'lag_1_PLUS_MINUS_pm_na', 'lag_1_PF_pm_na',
       'lag_1_START_POSITION_Missing', 'lag_1_START_POSITION_F'],
      dtype='<U56')

In [32]:
## 73.3 sec to boruta with 10 estimators on test data
## 406.5 sec to boruta with 10 estimators on train data

array([ 1, 32, 46,  1, 54,  1,  8,  5, 54, 33, 28, 27, 36,  1,  1,  1,  1,
       25, 29, 10,  8, 12, 34, 30,  3, 12, 14, 38, 40, 30, 36, 39,  1,  1,
        1,  7, 18, 20,  1,  1,  1,  1,  1,  1,  1,  4,  1, 23, 21, 17,  6,
        2,  1,  1, 19, 23, 75,  1,  1,  1, 75, 75, 75, 75, 75, 22, 75, 75,
       75, 75, 15, 75, 54, 75, 54, 15, 75, 75, 75, 75, 43, 75, 26, 75, 75,
       75, 75, 75, 75, 46, 46, 75, 75, 46, 54, 54, 46, 75, 41, 75, 54, 42,
       75, 46, 75, 54, 54, 75,  1,  1, 35, 54,  1, 12, 75, 75,  1,  1,  1])